In [148]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import locale
import mibian

In [2]:
# URL de la página a hacer scraping
url = 'https://www.meff.es/esp/Derivados-Financieros/Ficha/FIEM_MiniIbex_35'

# Realizar la petición HTTP GET a la página
response = requests.get(url)

In [3]:
def obtener_dataframe(response, tipo_tabla):
    """
    Realiza el web scraping y devuelve un dataframe con los datos obtenidos,
    dependiendo si el tipo de tabla es 'opciones' o 'futuros'.
    
    Args:
    - response: La respuesta HTTP obtenida.
    - tipo_tabla: Tipo de la tabla a buscar ('opciones' o 'futuros').
    
    Returns:
    - Un dataframe con los datos de la tabla.
    """
    # Determinar el ID de la tabla y si se necesita manejar el atributo data-tipo
    if tipo_tabla == 'opciones':
        id_tabla = 'tblOpciones'
        es_opcion = True
    elif tipo_tabla == 'futuros':
        id_tabla = 'Contenido_Contenido_tblFuturos'
        es_opcion = False
    else:
        print("Tipo de tabla no soportado.")
        return pd.DataFrame()
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', id=id_tabla)
        all_rows_data = []
        
        if table:
            rows = table.find_all('tr', class_='text-right')
            
            for row in rows:
                cells = row.find_all('td')
                row_data = [cell.text.strip() for cell in cells]
                
                if es_opcion:
                    data_tipo = row.get('data-tipo', 'No especificado')
                    row_data.insert(0, data_tipo)
                    
                all_rows_data.append(row_data)
                
            return pd.DataFrame(all_rows_data)
        else:
            print('No se encontró la tabla con el id especificado.')
            return pd.DataFrame()
    else:
        print('Error al realizar la petición HTTP:', response.status_code)
        return pd.DataFrame()

In [99]:
def tratar_dataframe(df, tipo_tabla):
    """
    Transforma el dataframe según si es de opciones o de futuros.
    
    Args:
    - df: Dataframe a transformar.
    - tipo_tabla: Tipo de la tabla ('opciones' o 'futuros').
    
    Returns:
    - Un dataframe transformado.
    """
    if tipo_tabla == 'opciones':
        # Especificar los nombres de columna para opciones
        df.columns = ['Class', 'Strike', 'Buy_ord', 'Buy_vol', 'Buy_price', 'Sell_price', 'Sell_vol', 'Sell_ord', 'Ult', 'Vol', 'Aper', 'Max.', 'Min.','Ant']
        df['Tipo'] = df['Class'].str[:3]
        df['Fecha'] = pd.to_datetime(df['Class'].str[3:], format='%Y%m%d').dt.strftime('%d-%m-%Y')
        df = df.drop(['Class'], axis=1)
        
        # Transformaciones adicionales para opciones
        df['Strike'] = df['Strike'].str.replace('.', '').str.replace(',', '.').astype(float)
        df['Ant'] = pd.to_numeric(df['Ant'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d-%m-%Y')
        
        # Seleccionando solo las columnas deseadas para opciones
        df = df.loc[:, ['Tipo', 'Fecha', 'Strike', 'Ant']]
        
    elif tipo_tabla == 'futuros':
        # Especificar los nombres de columna para futuros
        df.columns = ['Vencimiento', 'Tipo', 'Buy_ord', 'Buy_vol', 'Buy_price', 'Sell_price', 'Sell_vol', 'Sell_ord', 'Ult', 'Vol', 'Aper', 'Max.', 'Min.','Ant']
        
        # Configurar locale a español
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8' if locale.windows_locale is None else 'Spanish')
        df['Vencimiento'] = pd.to_datetime(df['Vencimiento'], format='%d %b %Y')
        df['Ant'] = pd.to_numeric(df['Ant'].str.replace('.', '').str.replace(',', '.'), errors='coerce')
        
        # Seleccionando solo las columnas deseadas para futuros
        df = df.loc[:, ['Vencimiento', 'Ant']]
        
    else:
        print("Tipo de tabla no soportado.")
        return pd.DataFrame()
    
    return df

In [106]:
# Opciones
df_opciones = obtener_dataframe(response, 'opciones')
df_opciones = tratar_dataframe(df_opciones, 'opciones')

In [101]:
# Futuros
df_futuros = obtener_dataframe(response, 'futuros')
df_futuros = tratar_dataframe(df_futuros, 'futuros')

In [110]:
# Filtrar para obtener solo las opciones de compra (calls) y de venta (puts)
df_calls = df_opciones[df_opciones['Tipo'] == 'OCE']
df_puts = df_opciones[df_opciones['Tipo'] == 'OPE']

# Crear un diccionario donde cada fecha es una clave y el valor es el DataFrame correspondiente
calls_por_fecha = {fecha: grupo for fecha, grupo in df_calls.groupby('Fecha')}
puts_por_fecha = {fecha: grupo for fecha, grupo in df_puts.groupby('Fecha')}

# Incluir estos diccionarios bajo las claves 'Call' y 'Put' en el diccionario 'resultados'
resultados = {
    'Call': calls_por_fecha,
    'Put': puts_por_fecha
}


In [116]:
df_futuros

,Vencimiento,Ant
0,2024-04-19,10523.7
1,2024-05-17,10489.0
2,2024-06-21,10496.0


In [118]:
price_sub = df_futuros.loc[0, 'Ant']
rfr = 0

dia_futuro = df_futuros.loc[0, 'Vencimiento']

In [119]:
dia_futuro

Timestamp('2024-04-19 00:00:00')

In [134]:
fechas = [dia for dia in resultados['Call'].keys()]

[Timestamp('2024-04-19 00:00:00'),
 Timestamp('2024-04-26 00:00:00'),
 Timestamp('2024-05-03 00:00:00'),
 Timestamp('2024-05-10 00:00:00'),
 Timestamp('2024-05-17 00:00:00'),
 Timestamp('2024-06-21 00:00:00'),
 Timestamp('2024-09-20 00:00:00'),
 Timestamp('2024-12-20 00:00:00'),
 Timestamp('2025-03-21 00:00:00'),
 Timestamp('2025-06-20 00:00:00'),
 Timestamp('2025-09-19 00:00:00'),
 Timestamp('2025-12-19 00:00:00'),
 Timestamp('2026-03-20 00:00:00'),
 Timestamp('2026-06-19 00:00:00')]

In [154]:
df_calls

,Tipo,Fecha,Strike,Ant
1,OCE,2024-04-19,8400.0,2123.0
2,OCE,2024-04-19,8500.0,2023.0
5,OCE,2024-04-19,8600.0,1923.0
6,OCE,2024-04-19,8700.0,1823.0
9,OCE,2024-04-19,8800.0,1723.0
...,...,...,...,...
1240,OCE,2026-06-19,10700.0,792.0
1241,OCE,2026-06-19,10800.0,751.0
1244,OCE,2026-06-19,10900.0,711.0
1245,OCE,2026-06-19,11000.0,673.0


In [165]:
for index, row in df_calls.iterrows():
    if dia_futuro == row['Fecha']:
        continue
    
    else:
        print('Ya es diferente')
        print(row)
    break

Ya es diferente
Tipo                      OCE
Fecha     2024-04-26 00:00:00
Strike                 9000.0
Ant                    1530.0
Name: 195, dtype: object


#### **Función de mibian.BS**

Dataframe opciones: Strike, Precio de la opción y la fecha.

Dataframe futuros: underlying price (subyacente) y fecha

 - **underlying_price:** Precio subyacente, es el precio del futuro MINI IBEX al vencimiento más próximo.
 
 - **call price:** Es el precio de la opción, columna 'ANT'

 - **interest rate:** 0

 - **days to expiration:** Diferencia entre la fecha actual y la de vencimiento

In [151]:
# Underlying price, strike price, interest rate, days until expiration
underlying_price = 1.4565
strike_price = 1.45
interest_rate = 1
days_to_expiration = 0
call_price = 0.0359

# Creamos un objeto para la opción de compra (call option)
c = mibian.BS([underlying_price, strike_price, interest_rate, days_to_expiration], callPrice=call_price)

# Obtenemos la volatilidad implícita de la opción de compra
implied_volatility = c.impliedVolatility


ZeroDivisionError: float division by zero

In [150]:
implied_volatility

19.22607421875